# piRNA Protein Pair Interaction network analysis

The objective of this project is to predict protein-protein interactions of piRNA.

In [1]:
from pathlib import Path

from pirna_ppi.utils import run_cmd, get_len1
from pirna_ppi.config import RAW_DATA_DIR, INTERIM_DATA_DIR, EXTERNAL_DATA_DIR

2025-12-25 09:31:43.261 | INFO     | pirna_ppi.config:<module>:11 - PROJ_ROOT path is: /home/gyk/project/pirna_ppi


## Configuration

In [2]:
# 00. environment config
colabfold_sif = Path("/home/gyk/sifs/colabfold_1.5.5-cuda12.2.2.sif")

# 01. Generate piRNA protein pair MSA
colabfold_db = Path("/data1/gyk/ColabFold_db")
protein_seqs = RAW_DATA_DIR / "piRNA_PPI_protein.fasta"
protein_pairs = INTERIM_DATA_DIR / "01.protein_pairs.fasta"
protein_pairs_msa = INTERIM_DATA_DIR / "01.protein_pairs_msa"

# 02. DCA Analysis
DCA_SCORE_DIR = INTERIM_DATA_DIR / "02.dca_scores"
DCA_ProtPair_cutoff = 0.12


## piRNA Protein Pair

In [ ]:
# Generate protein pairs
run_cmd(
    f"pixi run python -m pirna_ppi.protein_pairs --input-path {protein_seqs} --output-path {protein_pairs}",
    outputs=[protein_pairs]
)


2025-12-16 13:41:02.799 | INFO     | pirna_ppi.config:<module>:11 - PROJ_ROOT path is: /home/gyk/project/pirna_ppi


2025-12-16 13:41:03.307 | INFO     | __main__:main:62 - Reading FASTA: /home/gyk/project/pirna_ppi/data/raw/piRNA_PPI_protein.fasta
2025-12-16 13:41:03.423 | INFO     | __main__:main:76 - Loaded 274 proteins. Pairs to write: 37401
2025-12-16 13:41:03.423 | INFO     | __main__:main:86 - Writing pairs to: /home/gyk/project/pirna_ppi/data/interim/protein_pairs.fasta
2025-12-16 13:41:03.553 | SUCCESS  | __main__:main:90 - Protein pair generation complete.


100%|██████████| 37401/37401 [00:00<00:00, 289848.92it/s]


In [9]:
# Generate protein pairs MSA files
run_cmd(
    f"pixi run colabfold_search {protein_pairs} {colabfold_db} {protein_pairs_msa} --gpu 0 --max-accept 1000 --threads 20",
    outputs=[protein_pairs_msa],
    force=True
)

INFO:colabfold.mmseqs.search:Running mmseqs createdb /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa/query.fas /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa/qdb --shuffle 0 --dbtype 1


/home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa/qdb exists and will be overwritten
createdb /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa/query.fas /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa/qdb --shuffle 0 --dbtype 1 

Converting sequences
[=======
Time for merging to qdb_h: 0h 0m 0s 15ms
Time for merging to qdb: 0h 0m 0s 22ms
Database type: Aminoacid
Time for processing: 0h 0m 0s 220ms
search /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa/qdb /data1/gyk/ColabFold_db/uniref30_2302_db /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa/res /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa/tmp --threads 20 --num-iterations 3 --db-load-mode 0 -a -e 0.1 --max-seqs 10000 --prefilter-mode 0 --k-score 'seq:96,prof:80' 

prefilter /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa/qdb /data1/gyk/ColabFold_db/uniref30_2302_db.idx /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa/tmp/122

INFO:colabfold.mmseqs.search:Running mmseqs search /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa/qdb /data1/gyk/ColabFold_db/uniref30_2302_db /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa/res /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa/tmp --threads 20 --num-iterations 3 --db-load-mode 0 -a -e 0.1 --max-seqs 10000 --prefilter-mode 0 --k-score 'seq:96,prof:80'
Index was not built with `prefilter` support. Please rebuild the index with:
	createindex --index-subset 0
Traceback (most recent call last):
  File "/home/gyk/project/pirna_ppi/.pixi/envs/default/bin/colabfold_search", line 10, in <module>
    sys.exit(main())
  File "/home/gyk/project/pirna_ppi/.pixi/envs/default/lib/python3.10/site-packages/colabfold/mmseqs/search.py", line 468, in main
    mmseqs_search_monomer(
  File "/home/gyk/project/pirna_ppi/.pixi/envs/default/lib/python3.10/site-packages/colabfold/mmseqs/search.py", line 130, in mmseqs_search_monomer
    run_mmseqs(mmseq

CalledProcessError: Command 'pixi run colabfold_search /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs.fasta /data1/gyk/ColabFold_db /home/gyk/project/pirna_ppi/data/interim/01.protein_pairs_msa --gpu 0 --max-accept 1000 --threads 20' returned non-zero exit status 1.

## DCA Analysis

In [ ]:
# Run DCA
for a3m_file in protein_pairs_msa.glob("*a3m"):
    pair_name = a3m_file.stem
    first_gene_name = pair_name.split('_')[0]
    output_npz_file = DCA_SCORE_DIR / f"{pair_name}.npz"
    L1_length = get_len(first_gene_name, protein_seqs)
    print(f"Processing {pair_name} with L1={L1_length}")
    run_cmd(
        f"pixi run python -m pirna_ppi.run_dca {a3m_file} {output_npz_file} {L1_length}",
        outputs=[output_npz_file]
    )

In [ ]:
# DCA Score Filter
